In [ ]:
# API Key를 환경변수로 관리하기 위한 설정 파일

from dotenv import load_dotenv

# API Key 정보로드
load_dotenv()

import os
os.environ["LANGCHAIN_PROJECT"] = "langchain_study"

In [51]:
from langchain_community.llms import Ollama

llm_aya = Ollama(model="aya", temperature=0)
llm_llama3 = Ollama(model="llama3:instruct", temperature=0)
llm_gemma = Ollama(model="gemma", temperature=0)


In [70]:
from langchain_openai import ChatOpenAI

llm_gpt = ChatOpenAI(model="gpt-4o", temperature=0)

In [60]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """
    System : 너는 다음 Instruction 을 잘 수행하는 assistant 이다.
    Instruction : {instruction}
    """
)

In [53]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

In [54]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

In [72]:
chain_aya = prompt | llm_aya | output_parser
chain_llama3 = prompt | llm_llama3 | output_parser
chain_gemma = prompt | llm_gemma | output_parser
chain_gpt = prompt | llm_gpt | output_parser

chain_llms = RunnableParallel(aya=chain_aya, gemma=chain_gemma, llama3=chain_llama3, gpt=chain_gpt)

In [57]:
# chain_llms 실행 Test
responses = chain_llms.invoke({"instruction":"HOT6 라는 음료수에 대해 알려주세요."})

from pprint import pprint
pprint(responses)

{'aya': 'HOT6는 시원한 탄산수와 달콤한 과일 향이 조화를 이루는 독특한 음료입니다. 이 음료수는 상쾌한 맛과 함께 비타민을 '
        '함유하고 있어 건강에도 좋습니다. HOT6는 다양한 과일 향으로 출시되어 있으며, 각기 다른 독특한 맛을 제공합니다. 예를 '
        '들어, 자몽과 오렌지 향이 나는 HOT6는 상큼한 맛을, 복숭아와 하트 향이 나는 HOT6는 달콤하고 부드러운 맛을 '
        '제공합니다. 이 음료수는 특히 더운 날씨에 갈증 해소와 함께 과일 향을 즐기고 싶을 때 좋습니다. 또한, HOT6는 다양한 '
        '패키지 옵션으로 출시되어 있어 편리하게 즐길 수 있습니다.',
 'gemma': 'HOT6 라는 음료수는 아마도 HOT CHOCOLATE 라는 음료수를 의미할 것입니다. HOT CHOCOLATE는 '
          '초콜릿과 우유를 섞은 음료수입니다.',
 'llama3': "As your assistant, I'd be happy to provide information about "
           'HOT6.\n'
           '\n'
           'HOT6 is a popular Korean energy drink that has gained a '
           'significant following worldwide. Here are some key points about '
           'HOT6:\n'
           '\n'
           '**Taste:** HOT6 has a unique and refreshing taste profile that '
           'combines sweet and sour notes with a hint of citrus.\n'
           '\n'
           '**Ingredients:** The drink contains a blend of natural '
      

In [81]:
eval_prompt = PromptTemplate(
    template="""
    System : 
    너는 llm model들의 답변을 비교하고 평가하는 AI 이다.
    Instruction과 Responses 안의 각각의 llm별 응답을 
    정확성(Accuracy), 관련성(Relevance), 유창성(Fluency), 완전성(Completeness) 측면에서    
    분석하고 우열을 가려라.
    한국어로 답변해줘.

    Instruction : {instruction}
    Resonses : {responses}
    """,
    input_variables=["instruction", "responses"],
    #partial_variables={"instruction" : instruction},
)

In [74]:
chain_eval = eval_prompt | llm_gpt

In [78]:
chain_combinded = (
    {"responses" : chain_llms, "instruction" : RunnablePassthrough()}
    | chain_eval
)

In [79]:
instruction = "argoCD에 대해 알려주세요."

response = chain_combinded.invoke({"instruction":instruction})

In [80]:
print(response.content)

각 LLM 모델의 응답을 분석하고 평가해보겠습니다.

### aya
**장점:**
1. Argo CD의 주요 기능을 상세히 설명함.
2. 지속적인 배포, 환경 관리, 동기화 및 복제, 시각적 인터페이스, 작업 흐름 통합 등 다양한 기능을 다룸.
3. 구체적인 예시와 함께 설명하여 이해하기 쉽게 작성됨.

**단점:**
1. 다소 길고 복잡하게 느껴질 수 있음.
2. 일부 기능 설명이 중복되거나 불필요하게 자세할 수 있음.

### gemma
**장점:**
1. 간결하고 명확하게 주요 기능을 설명함.
2. 자동화된 배포, GitOps 모델, Rolling Deployment, 다양한 플랫폼 지원 등 핵심 기능을 잘 다룸.
3. 개발자와 DevOps 엔지니어에게 유용한 도구임을 강조함.

**단점:**
1. 기능 설명이 다소 간략하여 깊이 있는 이해를 제공하지 못할 수 있음.
2. 일부 기능 설명이 부족하거나 생략됨.

### llama3
**장점:**
1. GitOps, Kubernetes-native, Application-centric, Multi-environment support, Rollbacks and rollouts 등 주요 기능을 잘 설명함.
2. Argo CD의 이점과 사용 사례를 명확히 제시함.
3. 간결하면서도 핵심을 잘 짚어줌.

**단점:**
1. 일부 용어가 다소 기술적일 수 있어 초보자에게는 어려울 수 있음.
2. 한국어가 아닌 영어로 작성되어 있어 한국어 사용자에게는 불편할 수 있음.

### gpt
**장점:**
1. GitOps 기반 배포, 자동 동기화, 롤백 및 히스토리 관리, 다양한 인증 및 권한 관리, 웹 UI 및 CLI, 다양한 배포 전략 지원, 확장성 등 주요 기능을 잘 설명함.
2. Argo CD의 장점을 명확히 제시함.
3. 간결하면서도 핵심을 잘 짚어줌.

**단점:**
1. 일부 기능 설명이 다소 간략할 수 있음.
2. 기술적인 용어가 많아 초보자에게는 어려울 수 있음.

### 종합 평가
1. **aya**: 기